# Préparation des variables de mode de vie

In [1]:
library(tidyverse)
library(factoextra)

# Charger les données
df_pilot1_2022 <- readRDS("../../../data/qc2022/preparation_donnees/01_pilot1_2022.rds")

# Préparation du jeu de données pour les variables de style
variables_mode_de_vie <- c(
  "id",
  #"postal_code", # À transformer en rural, urbain, région, banlieue
  "ses_dwelling_App",
  "ses_dwelling_Loft",
  "ses_dwelling_Condo",
  "ses_dwelling_Tour",
  "ses_dwelling_detachedHouse",
  "ses_dwelling_townHouse",
  "ses_dwelling_Coop",
  "ses_dwelling_HLM",
  "ses_dwelling_Mobile",
  "ses_dwelling_Other",
  "act_transport_Car",
  "act_transport_SUV",
  "act_transport_Moto",
  "act_transport_Walk",
  "act_transport_Bicycle",
  "act_transport_PublicTransportation",
  "vehicule_4x4",
  "vehicule_Berline",
  "vehicule_Cabriolet",
  "vehicule_PickUp",
  "vehicule_Van",
  "vehicule_luxury",
  "vehicule_sport",
  "vehicule_electric",
  "vehicule_VUS",
  "vehicule_other",
  "vehicule_noCar"
)

df_mode_de_vie <- df_pilot1_2022 %>%
  select(all_of(variables_mode_de_vie))

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.1     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.1
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa



In [2]:
library(dplyr)

df_mode_de_vie %>%
  select(starts_with("vehicule")) %>%
  summarise(across(everything(), sum, na.rm = TRUE))

df_mode_de_vie %>%
  select(starts_with("act_transport")) %>%
  summarise(across(everything(), sum, na.rm = TRUE))

Warning message:
"There was 1 warning in `summarise()`.
i In argument: `across(everything(), sum, na.rm = TRUE)`.
Caused by warning:
! The `...` argument of `across()` is deprecated as of dplyr 1.1.0.
Supply arguments directly to `.fns` through an anonymous function instead.

  # Previously
  across(a:b, mean, na.rm = TRUE)

  # Now
  across(a:b, \(x) mean(x, na.rm = TRUE))"


vehicule_4x4,vehicule_Berline,vehicule_Cabriolet,vehicule_PickUp,vehicule_Van,vehicule_luxury,vehicule_sport,vehicule_electric,vehicule_VUS,vehicule_other,vehicule_noCar
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
64,520,13,43,54,20,28,96,396,66,200


act_transport_Car,act_transport_SUV,act_transport_Moto,act_transport_Walk,act_transport_Bicycle,act_transport_PublicTransportation
<int>,<int>,<int>,<int>,<int>,<int>
913,183,8,188,53,155


In [3]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_mode_de_vie) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_mode_de_vie[[v]]))
    cat("\n\n")
  }
}

Table pour la variable: ses_dwelling_App 

   0    1 
1201  299 


Table pour la variable: ses_dwelling_Loft 

   0    1 
1492    8 


Table pour la variable: ses_dwelling_Condo 

   0    1 
1334  166 


Table pour la variable: ses_dwelling_Tour 

   0    1 
1463   37 


Table pour la variable: ses_dwelling_detachedHouse 

  0   1 
784 716 


Table pour la variable: ses_dwelling_townHouse 

   0    1 
1419   81 


Table pour la variable: ses_dwelling_Coop 

   0    1 
1493    7 


Table pour la variable: ses_dwelling_HLM 

   0    1 
1488   12 


Table pour la variable: ses_dwelling_Mobile 

   0    1 
1491    9 


Table pour la variable: ses_dwelling_Other 

   0    1 
1335  165 


Table pour la variable: act_transport_Car 

  0   1 
587 913 


Table pour la variable: act_transport_SUV 

   0    1 
1317  183 


Table pour la variable: act_transport_Moto 

   0    1 
1492    8 


Table pour la variable: act_transport_Walk 

   0    1 
1312  188 


Table pour la variable: act_transport_

### Transformations

- Regroupement de ses_dwelling
  - ses_dwelling_Condo : ses_dwelling_Condo + ses_dwelling_Loft + ses_dwelling_Tour
  - ses_dwelling_Other : ses_dwelling_Other + ses_dwelling_Coop + ses_dwelling_HLM + ses_dwelling_Mobile

- Regroupement des vehicules
  - vehicule_ToutTerrain : vehicule_4x4	+ vehicule_PickUp
  - vehicle_Voiture : vehicule_Berline + vehicule_Cabriolet + vehicule_luxury + vehicule_sport
  - vehicule_Van
  - vehicule_electric
  - vehicule_VUS
  - vehicule_other
  - vehicule_noCar

- Regroupement des act_transport
  - act_transport_Car : act_transport_Car + act_transport_Moto (parce qu'il y avraiment trop peu de moto, assigner la classe la plus populeuse)
  - Les autres ne sont pas regroupés

In [4]:
# Regroupement de ses_dwelling
#-----------------------------

df_mode_de_vie <- df_mode_de_vie %>%

  mutate(
    ses_dwelling_Condo = if_else(
      ses_dwelling_Loft == 1 | ses_dwelling_Tour == 1,
      1,
      ses_dwelling_Condo
    ),
    ses_dwelling_Other = if_else(
      ses_dwelling_Coop == 1 | ses_dwelling_HLM == 1 | ses_dwelling_Mobile == 1,
      1,
      ses_dwelling_Other
    )
  ) %>%
  select(
    -ses_dwelling_Loft,
    -ses_dwelling_Tour,
    -ses_dwelling_Coop,
    -ses_dwelling_HLM,
    -ses_dwelling_Mobile
  )

In [5]:
# Regroupement de vehicule
#-------------------------

df_mode_de_vie <- df_mode_de_vie %>%

  mutate(
    vehicule_ToutTerrain = if_else(
      vehicule_4x4 == 1 | vehicule_PickUp == 1,
      1,
      0
    ),
    vehicule_Voiture = if_else(
      vehicule_Berline == 1 | vehicule_Cabriolet == 1 | vehicule_luxury == 1 | vehicule_sport,
      1,
      0
    )
  ) %>%
  select(
    -vehicule_4x4,
    -vehicule_PickUp,
    -vehicule_Berline,
    -vehicule_Cabriolet,
    -vehicule_luxury,
    -vehicule_sport
  )

In [6]:
# Regroupement de act_transport
#------------------------------

df_mode_de_vie <- df_mode_de_vie %>%

  mutate(
    act_transport_Car = if_else(
      act_transport_Moto == 1,
      1,
      act_transport_Car
    )) %>%
  select(
    -act_transport_Moto
  )

In [7]:
variables_mode_de_vie_clust <- c(
  "id",
  #"postal_code", # À transformer en rural, urbain, région, banlieue
  "ses_dwelling_App",
  "ses_dwelling_Condo",
  "ses_dwelling_detachedHouse",
  "ses_dwelling_townHouse",
  "ses_dwelling_Other",
  "act_transport_Car",
  "act_transport_SUV",
  "act_transport_Walk",
  "act_transport_Bicycle",
  "act_transport_PublicTransportation",
  "vehicule_ToutTerrain",
  "vehicule_Van",
  "vehicule_Voiture",
  "vehicule_electric",
  "vehicule_VUS",
  "vehicule_other",
  "vehicule_noCar"
)

In [8]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_mode_de_vie_clust) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_mode_de_vie[[v]]))
    cat("\n\n")
  }
}

Table pour la variable: ses_dwelling_App 

   0    1 
1201  299 


Table pour la variable: ses_dwelling_Condo 

   0    1 
1289  211 


Table pour la variable: ses_dwelling_detachedHouse 

  0   1 
784 716 


Table pour la variable: ses_dwelling_townHouse 

   0    1 
1419   81 


Table pour la variable: ses_dwelling_Other 

   0    1 
1307  193 


Table pour la variable: act_transport_Car 

  0   1 
579 921 


Table pour la variable: act_transport_SUV 

   0    1 
1317  183 


Table pour la variable: act_transport_Walk 

   0    1 
1312  188 


Table pour la variable: act_transport_Bicycle 

   0    1 
1447   53 


Table pour la variable: act_transport_PublicTransportation 

   0    1 
1345  155 


Table pour la variable: vehicule_ToutTerrain 

   0    1 
1393  107 


Table pour la variable: vehicule_Van 

   0    1 
1446   54 


Table pour la variable: vehicule_Voiture 

  0   1 
919 581 


Table pour la variable: vehicule_electric 

   0    1 
1404   96 


Table pour la variable: ve

In [9]:
# Sauvegarder les données préparées
saveRDS(df_mode_de_vie, file = "../../../data/qc2022/preparation_donnees/02_pilot1_2022_mode_de_vie.rds")